# Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np
import time
import warnings

from tqdm.notebook import tqdm

from sklearn import preprocessing

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score
from sklearn.model_selection import cross_val_predict

from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [2]:
start = time.time()
warnings.filterwarnings("ignore")

# Обработка данных

In [3]:
data = pd.read_csv(r"Data/work_table.csv")
spprices = pd.read_csv(r"Data/spprices.csv")

In [4]:
columns = []
for i in np.arange(len(spprices.columns)-9):
    columns.append(f'{2010+i}')
spprofit = pd.DataFrame(columns = columns)
row = []
for i in np.arange(len(spprofit.columns)):
    row.append((spprices.iloc[0,i+9] / spprices.iloc[0,i+4] - 1)*100)
spprofit.loc[0] = row

In [5]:
spprofit

,2010,2011,2012,2013,2014,2015,2016
0,74.831508,55.56623,66.244274,65.275226,52.063622,19.945234,93.936163


In [6]:
choice = []
for i in np.arange(len(data.index)-len(data[data['Year of examination'] == 2021].index)):
    if data.Result[i] > spprofit[f'{data["Year of examination"][i]}'][0]:
        choice.append(1)
    else:
        choice.append(0)
for i in np.arange(len(data[data['Year of examination'] == 2021].index)):
    choice.append(np.nan)

In [7]:
data['Choice'] = choice
data['Choice'].value_counts()

1.0    1594
0.0    1413
Name: Choice, dtype: int64

In [8]:
data1 = data[pd.isna(data['EBITDA 1']) == False]
data1 = data1.reset_index(drop = True)
num_fut1 = len(data1[pd.isna(data1['Choice']) == True].index)

In [9]:
cat_cols1 = ['Sector', 'Sub-sector']
num_cols1 = data1.columns[5:-2].tolist()
target_col = 'Choice'

In [10]:
num_data1 = data1[num_cols1]

In [11]:
for i in np.arange(len(num_cols1)):
    num_data1[num_cols1[i]] = round((num_data1[num_cols1[i]] - min(num_data1[num_cols1[i]])) / (max(num_data1[num_cols1[i]]) - min(num_data1[num_cols1[i]])), 6)

In [12]:
cat_data1 = pd.get_dummies(data1[cat_cols1])
cat_data1 = pd.DataFrame(cat_data1)

In [13]:
full_data1 = num_data1.join(cat_data1)

In [14]:
X1_fut = full_data1.iloc[len(full_data1.index)-num_fut1:]

y1 = data1['Choice'][:len(full_data1.index)-num_fut1]
X1 = full_data1.iloc[:len(full_data1.index)-num_fut1]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X1.values, y1.values,
                                                      train_size=0.8,
                                                      random_state=42)

In [16]:
data_for_catboost1 = data.drop(columns = ['Company', 'Ticker', 'Year of examination', 'Result', 'Choice'])
data_for_catboost1 = data_for_catboost1[pd.isna(data['EBITDA 1']) == False]
data_for_catboost1 = data_for_catboost1.reset_index(drop = True)
data_for_catboost1 = data_for_catboost1.iloc[:len(data_for_catboost1.index)-num_fut1]

In [17]:
for i in np.arange(2,len(data_for_catboost1.columns)):
    data_for_catboost1[data_for_catboost1.columns[i]] = round((data_for_catboost1[data_for_catboost1.columns[i]] - min(data_for_catboost1[data_for_catboost1.columns[i]])) / (max(data_for_catboost1[data_for_catboost1.columns[i]]) - min(data_for_catboost1[data_for_catboost1.columns[i]])), 6)

In [18]:
X_full = data_for_catboost1[:len(data_for_catboost1.index)-num_fut1]
y = data1['Choice'][:len(data_for_catboost1.index)-num_fut1]

X_full_cat = data_for_catboost1[len(full_data1.index)-num_fut1:]

cat_indices = np.array([0,1])

In [19]:
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_full.values, y.values,
                                                      train_size=0.8,
                                                      random_state=42)

# Отбор моделей

In [20]:
models = [
    LogisticRegression(),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    ExtraTreesClassifier(),
    LinearDiscriminantAnalysis(),
    XGBClassifier(),
    QuadraticDiscriminantAnalysis()
]

In [21]:
log_param_grid = {
    'C' : np.arange(10000000,1000000001, 10000000),
    'penalty' : ['l1', 'l2'],
    'solver' : ['lbfgs', 'saga']
}
kneighbors_param_grid = {
    'n_neighbors' : np.arange(1, 11),
    'metric' : ['minkowski', 'manhattan', 'euclidean'],
    'weights' : ['uniform', 'distance']
}
decision_tree_param_grid = {
    'max_depth' : np.arange(2, 21, 2),
    'min_samples_leaf': np.arange(2,6),
    'min_samples_split': np.arange(2,6)
}
random_forest_param_grid = {
    'n_estimators' : np.arange(50, 501, 50),
    'min_samples_leaf': np.arange(2,6),
    'min_samples_split': np.arange(2,6)
}
extra_trees_param_grid = {
    'n_estimators' : np.arange(50, 501, 50),
    'min_samples_leaf': np.arange(2,6),
    'min_samples_split': np.arange(2,6)
}
lin_discriminant_analysis_param_grid = {
    'solver' : ['svd', 'eigen', 'lsqr']
}
xgb_param_grid = {
    'n_estimators' : np.arange(100, 501, 50),
    'max_depth' : np.arange(3, 10, 3),
    'eta' : np.arange(0.05, 0.31, 0.05)
}
param_grids = [log_param_grid, kneighbors_param_grid, decision_tree_param_grid,
               random_forest_param_grid, extra_trees_param_grid,
               lin_discriminant_analysis_param_grid, xgb_param_grid]

In [22]:
better_models = []
i=0
for model, param_grid in zip(tqdm(models[:-1]), param_grids):   
    search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, refit=True, scoring='precision')
    search.fit(X_train, y_train)
    i+=1
    print(f'{i}:')
    print(search.best_params_)

    best_params = []
    for key, con in search.best_params_.items():
        temp_list = [key, con]
        best_params.append(temp_list)
    par1 = best_params[0][1]
    try:
        par2 = best_params[1][1]
        par3 = best_params[2][1]
    except Exception:
        pass
    if i==1:
        better_models.append(LogisticRegression(C=par1, penalty=par2, solver=par3))
    elif i==2:
        better_models.append(KNeighborsClassifier(metric=par1, n_neighbors=par2, weights=par3))
    elif i==3:
        better_models.append(DecisionTreeClassifier(max_depth=par1, min_samples_leaf=par2, min_samples_split=par3))
    elif i==4:
        better_models.append(RandomForestClassifier(min_samples_leaf=par1, min_samples_split=par2, n_estimators=par3))
    elif i==5:
        better_models.append(ExtraTreesClassifier(min_samples_leaf=par1, min_samples_split=par2, n_estimators=par3))
    elif i==6:
        better_models.append(LinearDiscriminantAnalysis(solver=par1))
    elif i==7:
        better_models.append(XGBClassifier(eta=par1, max_depth=par2, n_estimators=par3, verbosity=0))
    y_test_predicted = search.predict_proba(X_test)[:,1]
    
    test_auc = roc_auc_score(y_test, y_test_predicted)    
    
    y_test_predicted = search.predict(X_test)
    
    test_accuracy = accuracy_score(y_test, y_test_predicted)
    test_precision = precision_score(y_test, y_test_predicted)
    
    print('\troc_auc_score:', test_auc)
    print('\taccuracy:', test_accuracy)
    print('\tprecision:', test_precision)
    
better_models.append(models[-1])

1:
{'C': 340000000, 'penalty': 'l2', 'solver': 'lbfgs'}
	roc_auc_score: 0.7572206917661464
	accuracy: 0.6713286713286714
	precision: 0.6741214057507987
2:
{'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
	roc_auc_score: 0.7825528007346187
	accuracy: 0.7202797202797203
	precision: 0.7939914163090128
3:
{'max_depth': 12, 'min_samples_leaf': 5, 'min_samples_split': 3}
	roc_auc_score: 0.6850260177532905
	accuracy: 0.6363636363636364
	precision: 0.6731517509727627
4:
{'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 350}
	roc_auc_score: 0.8466054484236303
	accuracy: 0.7552447552447552
	precision: 0.7288629737609329
5:
{'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 350}
	roc_auc_score: 0.8675237220691766
	accuracy: 0.7954545454545454
	precision: 0.7795031055900621
6:
{'solver': 'svd'}
	roc_auc_score: 0.741010101010101
	accuracy: 0.6590909090909091
	precision: 0.6677631578947368
[17:25:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_re

In [23]:
print(better_models)

[LogisticRegression(C=340000000), KNeighborsClassifier(metric='manhattan', n_neighbors=2), DecisionTreeClassifier(max_depth=12, min_samples_leaf=5, min_samples_split=3), RandomForestClassifier(min_samples_leaf=2, min_samples_split=4,
                       n_estimators=350), ExtraTreesClassifier(min_samples_leaf=2, n_estimators=350), LinearDiscriminantAnalysis(), XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              eta=0.15000000000000002, gamma=None, gpu_id=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=9,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate

In [24]:
i=0
best_prec = 0
for model in tqdm(better_models):
    model.fit(X_train, y_train)
    
    y_train_predicted = model.predict_proba(X_train)[:,1]
    y_test_predicted = model.predict_proba(X_test)[:,1]

    test_auc = roc_auc_score(y_test, y_test_predicted)    
    
    y_train_predicted = model.predict(X_train)
    y_test_predicted = model.predict(X_test) 
    
    test_acc = accuracy_score(y_test, y_test_predicted)

    test_prec = precision_score(y_test, y_test_predicted)
    
    if test_prec>best_prec:
        best_model = model
        best_prec = test_prec
    
    i+=1
    print(f'{i}:')
    print('\troc_auc_score: ', test_auc)
    print('\taccuracy: ', test_acc)
    print('\tprecision: ', test_prec)

1:
	roc_auc_score:  0.7572206917661464
	accuracy:  0.6713286713286714
	precision:  0.6741214057507987
2:
	roc_auc_score:  0.7825528007346187
	accuracy:  0.7202797202797203
	precision:  0.7939914163090128
3:
	roc_auc_score:  0.6944658708295073
	accuracy:  0.6416083916083916
	precision:  0.6796875
4:
	roc_auc_score:  0.8452525252525251
	accuracy:  0.7604895104895105
	precision:  0.7352941176470589
5:
	roc_auc_score:  0.8704254667891032
	accuracy:  0.8006993006993007
	precision:  0.7832817337461301
6:
	roc_auc_score:  0.741010101010101
	accuracy:  0.6590909090909091
	precision:  0.6677631578947368
7:
	roc_auc_score:  0.8409794918885827
	accuracy:  0.7552447552447552
	precision:  0.7357357357357357
8:
	roc_auc_score:  0.7022834404652586
	accuracy:  0.5961538461538461
	precision:  0.6853932584269663



# Стекинг

In [25]:
def compute_meta_feature(model, X_train, X_test, y_train, cv):
    try:
        train_answers = cross_val_predict(model, X_train, y_train, cv=cv, method='predict_proba')[:, 1]
        model.fit(X_train, y_train)
        return train_answers, model.predict_proba(X_test)[:, 1]
    
    except Exception:
        train_answers = cross_val_predict(model, X_train, y_train, cv=cv, method='predict')
        model.fit(X_train, y_train)
        return train_answers, model.predict(X_test)

def stacking(X_train, X_test, y_train, y_test=None, boo=True):
    smodels = []
    smodels.append(KNeighborsClassifier())
    smodels.append(LogisticRegression())
    smodels.append(RandomForestClassifier(max_depth=3, n_estimators=50))
    smodels.append(RandomForestClassifier(max_depth=5, n_estimators=50))
    smodels.append(DecisionTreeClassifier())
    smodels.append(ExtraTreesClassifier())
    smodels.append(QuadraticDiscriminantAnalysis())
    
    meta_features_train = np.zeros((X_train.shape[0], 0))
    meta_features_test = np.zeros((X_test.shape[0], 0))
    
    for model in tqdm(smodels):
        train, test = compute_meta_feature(model, X_train, X_test, y_train, 5)
        meta_features_train = np.append(meta_features_train, train.reshape((train.size, 1)), axis=1)
        meta_features_test = np.append(meta_features_test, test.reshape((test.size, 1)), axis=1)
    
    stacking_model = LogisticRegression()
    stacking_model.fit(meta_features_train, y_train)

    y_train_predicted = stacking_model.predict_proba(meta_features_train)[:, 1]
    y_test_predicted = stacking_model.predict_proba(meta_features_test)[:, 1]

    y_train_predict = stacking_model.predict(meta_features_train)
    y_test_predict = stacking_model.predict(meta_features_test)

    if not boo:
        return y_test_predict   
    
    else:
        train_auc = roc_auc_score(y_train, y_train_predicted)
        test_auc = roc_auc_score(y_test, y_test_predicted)

        train_acc = accuracy_score(y_train, y_train_predict)
        test_acc = accuracy_score(y_test, y_test_predict)

        train_precision = precision_score(y_train, y_train_predict)
        test_precision = precision_score(y_test, y_test_predict)

        print(train_auc, test_auc)
        print(train_acc, test_acc)
        print(train_precision, test_precision)
    
        return test_precision

In [26]:
stack_prec = stacking(X_train, X_test, y_train, y_test, True)


0.8534844282014539 0.8719314355677992
0.7745183887915937 0.8006993006993007
0.7877358490566038 0.7815384615384615


In [27]:
stack_prec

0.7815384615384615

In [28]:
bm = True
if stack_prec>best_prec:
    best_prec = stack_prec
    bm = False

# Бустинг

In [29]:
cat_param_grid = {
    'n_estimators' : np.arange(200, 1001, 100),
    'depth' : np.arange(3, 10, 3),
}

In [30]:
bcm = False
for item in tqdm(cat_param_grid['n_estimators']):
    for it in cat_param_grid['depth']:
        catboosting_model = CatBoostClassifier(n_estimators=item, 
                                             depth=it, 
                                             cat_features=cat_indices
                                             )

        catboosting_model.fit(X_train_cat, y_train_cat)

        y_test_predicted = catboosting_model.predict_proba(X_test_cat)[:, 1]

        y_test_predict = catboosting_model.predict(X_test_cat)

        test_auc = roc_auc_score(y_test_cat, y_test_predicted)

        test_acc = accuracy_score(y_test_cat, y_test_predict)

        test_prec = precision_score(y_test_cat, y_test_predict)
        print(test_auc)
        print(test_acc)
        print(test_prec)
        if test_prec>best_prec:
            best_pars = {
                'n_estimators' : item,
                'depth' : it,
                }
            best_cat_model = catboosting_model
            bcm = True

Learning rate set to 0.059587
0:	learn: 0.6822767	total: 328ms	remaining: 1m 5s
1:	learn: 0.6712604	total: 346ms	remaining: 34.2s
2:	learn: 0.6635338	total: 381ms	remaining: 25s
3:	learn: 0.6558951	total: 406ms	remaining: 19.9s
4:	learn: 0.6493292	total: 422ms	remaining: 16.5s
5:	learn: 0.6423860	total: 449ms	remaining: 14.5s
6:	learn: 0.6371343	total: 484ms	remaining: 13.3s
7:	learn: 0.6304459	total: 522ms	remaining: 12.5s
8:	learn: 0.6265209	total: 574ms	remaining: 12.2s
9:	learn: 0.6234881	total: 612ms	remaining: 11.6s
10:	learn: 0.6183114	total: 649ms	remaining: 11.1s
11:	learn: 0.6143827	total: 701ms	remaining: 11s
12:	learn: 0.6116651	total: 745ms	remaining: 10.7s
13:	learn: 0.6084983	total: 807ms	remaining: 10.7s
14:	learn: 0.6060374	total: 860ms	remaining: 10.6s
15:	learn: 0.6020320	total: 892ms	remaining: 10.3s
16:	learn: 0.5978146	total: 940ms	remaining: 10.1s
17:	learn: 0.5942853	total: 988ms	remaining: 9.99s
18:	learn: 0.5907528	total: 1.02s	remaining: 9.72s
19:	learn: 0.58

162:	learn: 0.4758856	total: 4.88s	remaining: 1.11s
163:	learn: 0.4746602	total: 4.91s	remaining: 1.08s
164:	learn: 0.4740228	total: 4.93s	remaining: 1.04s
165:	learn: 0.4737513	total: 4.95s	remaining: 1.01s
166:	learn: 0.4733324	total: 4.97s	remaining: 983ms
167:	learn: 0.4726438	total: 5s	remaining: 952ms
168:	learn: 0.4720512	total: 5.02s	remaining: 921ms
169:	learn: 0.4712247	total: 5.04s	remaining: 889ms
170:	learn: 0.4704119	total: 5.07s	remaining: 859ms
171:	learn: 0.4700329	total: 5.09s	remaining: 828ms
172:	learn: 0.4694430	total: 5.1s	remaining: 797ms
173:	learn: 0.4685301	total: 5.13s	remaining: 766ms
174:	learn: 0.4680331	total: 5.14s	remaining: 735ms
175:	learn: 0.4674463	total: 5.16s	remaining: 704ms
176:	learn: 0.4672727	total: 5.18s	remaining: 673ms
177:	learn: 0.4669735	total: 5.2s	remaining: 643ms
178:	learn: 0.4661288	total: 5.23s	remaining: 614ms
179:	learn: 0.4653403	total: 5.25s	remaining: 584ms
180:	learn: 0.4648074	total: 5.27s	remaining: 554ms
181:	learn: 0.464

121:	learn: 0.3681745	total: 7.35s	remaining: 4.7s
122:	learn: 0.3670119	total: 7.41s	remaining: 4.64s
123:	learn: 0.3654130	total: 7.47s	remaining: 4.58s
124:	learn: 0.3643027	total: 7.53s	remaining: 4.52s
125:	learn: 0.3631604	total: 7.6s	remaining: 4.46s
126:	learn: 0.3623553	total: 7.65s	remaining: 4.4s
127:	learn: 0.3614987	total: 7.71s	remaining: 4.34s
128:	learn: 0.3606449	total: 7.78s	remaining: 4.28s
129:	learn: 0.3590995	total: 7.83s	remaining: 4.22s
130:	learn: 0.3575050	total: 7.91s	remaining: 4.17s
131:	learn: 0.3563646	total: 7.96s	remaining: 4.1s
132:	learn: 0.3556096	total: 8.01s	remaining: 4.04s
133:	learn: 0.3543290	total: 8.08s	remaining: 3.98s
134:	learn: 0.3523820	total: 8.14s	remaining: 3.92s
135:	learn: 0.3506506	total: 8.19s	remaining: 3.85s
136:	learn: 0.3488608	total: 8.25s	remaining: 3.79s
137:	learn: 0.3469905	total: 8.3s	remaining: 3.73s
138:	learn: 0.3452114	total: 8.35s	remaining: 3.66s
139:	learn: 0.3434844	total: 8.41s	remaining: 3.61s
140:	learn: 0.342

80:	learn: 0.2703927	total: 20.3s	remaining: 29.9s
81:	learn: 0.2682695	total: 20.6s	remaining: 29.6s
82:	learn: 0.2666121	total: 20.8s	remaining: 29.4s
83:	learn: 0.2653067	total: 21s	remaining: 29s
84:	learn: 0.2622619	total: 21.2s	remaining: 28.7s
85:	learn: 0.2603519	total: 21.5s	remaining: 28.5s
86:	learn: 0.2566585	total: 21.8s	remaining: 28.3s
87:	learn: 0.2542524	total: 22s	remaining: 28s
88:	learn: 0.2524976	total: 22.3s	remaining: 27.8s
89:	learn: 0.2514414	total: 22.5s	remaining: 27.5s
90:	learn: 0.2484472	total: 22.8s	remaining: 27.3s
91:	learn: 0.2473949	total: 23s	remaining: 27s
92:	learn: 0.2456468	total: 23.3s	remaining: 26.8s
93:	learn: 0.2433828	total: 23.5s	remaining: 26.5s
94:	learn: 0.2424352	total: 23.7s	remaining: 26.2s
95:	learn: 0.2406591	total: 24s	remaining: 26s
96:	learn: 0.2387984	total: 24.2s	remaining: 25.7s
97:	learn: 0.2373938	total: 24.5s	remaining: 25.5s
98:	learn: 0.2354917	total: 24.6s	remaining: 25.1s
99:	learn: 0.2329562	total: 24.8s	remaining: 24

45:	learn: 0.5591943	total: 1.01s	remaining: 5.61s
46:	learn: 0.5580943	total: 1.04s	remaining: 5.61s
47:	learn: 0.5566867	total: 1.06s	remaining: 5.57s
48:	learn: 0.5556190	total: 1.08s	remaining: 5.54s
49:	learn: 0.5546458	total: 1.1s	remaining: 5.5s
50:	learn: 0.5533548	total: 1.12s	remaining: 5.47s
51:	learn: 0.5528548	total: 1.15s	remaining: 5.48s
52:	learn: 0.5523661	total: 1.17s	remaining: 5.46s
53:	learn: 0.5516350	total: 1.2s	remaining: 5.45s
54:	learn: 0.5509433	total: 1.23s	remaining: 5.46s
55:	learn: 0.5505193	total: 1.25s	remaining: 5.45s
56:	learn: 0.5497291	total: 1.28s	remaining: 5.44s
57:	learn: 0.5488400	total: 1.3s	remaining: 5.43s
58:	learn: 0.5479856	total: 1.32s	remaining: 5.4s
59:	learn: 0.5475270	total: 1.34s	remaining: 5.36s
60:	learn: 0.5465349	total: 1.37s	remaining: 5.36s
61:	learn: 0.5460751	total: 1.4s	remaining: 5.36s
62:	learn: 0.5452753	total: 1.42s	remaining: 5.34s
63:	learn: 0.5445590	total: 1.44s	remaining: 5.32s
64:	learn: 0.5439840	total: 1.46s	rem

...

326:	learn: 0.2823324	total: 1m 19s	remaining: 2m 43s
327:	learn: 0.2819494	total: 1m 19s	remaining: 2m 43s
328:	learn: 0.2815607	total: 1m 19s	remaining: 2m 42s
329:	learn: 0.2809900	total: 1m 20s	remaining: 2m 42s
330:	learn: 0.2806466	total: 1m 20s	remaining: 2m 42s
331:	learn: 0.2800634	total: 1m 20s	remaining: 2m 42s
332:	learn: 0.2797265	total: 1m 20s	remaining: 2m 42s
333:	learn: 0.2793602	total: 1m 21s	remaining: 2m 41s
334:	learn: 0.2788063	total: 1m 21s	remaining: 2m 41s
335:	learn: 0.2782473	total: 1m 21s	remaining: 2m 41s
336:	learn: 0.2775383	total: 1m 21s	remaining: 2m 41s
337:	learn: 0.2770743	total: 1m 22s	remaining: 2m 40s
338:	learn: 0.2765313	total: 1m 22s	remaining: 2m 40s
339:	learn: 0.2761038	total: 1m 22s	remaining: 2m 40s
340:	learn: 0.2755334	total: 1m 22s	remaining: 2m 40s
341:	learn: 0.2752108	total: 1m 23s	remaining: 2m 39s
342:	learn: 0.2748247	total: 1m 23s	remaining: 2m 39s
343:	learn: 0.2745767	total: 1m 23s	remaining: 2m 39s
344:	learn: 0.2740637	total:

478:	learn: 0.2162114	total: 1m 56s	remaining: 2m 6s
479:	learn: 0.2159482	total: 1m 56s	remaining: 2m 6s
480:	learn: 0.2154887	total: 1m 56s	remaining: 2m 6s
481:	learn: 0.2151877	total: 1m 57s	remaining: 2m 5s
482:	learn: 0.2147590	total: 1m 57s	remaining: 2m 5s
483:	learn: 0.2144373	total: 1m 57s	remaining: 2m 5s
484:	learn: 0.2139677	total: 1m 57s	remaining: 2m 5s
485:	learn: 0.2136701	total: 1m 58s	remaining: 2m 4s
486:	learn: 0.2133621	total: 1m 58s	remaining: 2m 4s
487:	learn: 0.2129602	total: 1m 58s	remaining: 2m 4s
488:	learn: 0.2124171	total: 1m 58s	remaining: 2m 4s
489:	learn: 0.2120716	total: 1m 59s	remaining: 2m 4s
490:	learn: 0.2118517	total: 1m 59s	remaining: 2m 3s
491:	learn: 0.2117112	total: 1m 59s	remaining: 2m 3s
492:	learn: 0.2113062	total: 1m 59s	remaining: 2m 3s
493:	learn: 0.2106489	total: 2m	remaining: 2m 2s
494:	learn: 0.2103620	total: 2m	remaining: 2m 2s
495:	learn: 0.2098908	total: 2m	remaining: 2m 2s
496:	learn: 0.2096473	total: 2m	remaining: 2m 2s
497:	lear

632:	learn: 0.1669641	total: 2m 33s	remaining: 1m 29s
633:	learn: 0.1666965	total: 2m 33s	remaining: 1m 28s
634:	learn: 0.1663630	total: 2m 34s	remaining: 1m 28s
635:	learn: 0.1661295	total: 2m 34s	remaining: 1m 28s
636:	learn: 0.1659480	total: 2m 34s	remaining: 1m 28s
637:	learn: 0.1657448	total: 2m 34s	remaining: 1m 27s
638:	learn: 0.1655293	total: 2m 35s	remaining: 1m 27s
639:	learn: 0.1652861	total: 2m 35s	remaining: 1m 27s
640:	learn: 0.1650922	total: 2m 35s	remaining: 1m 27s
641:	learn: 0.1646541	total: 2m 35s	remaining: 1m 26s
642:	learn: 0.1643710	total: 2m 36s	remaining: 1m 26s
643:	learn: 0.1641276	total: 2m 36s	remaining: 1m 26s
644:	learn: 0.1639505	total: 2m 36s	remaining: 1m 26s
645:	learn: 0.1636361	total: 2m 36s	remaining: 1m 25s
646:	learn: 0.1633456	total: 2m 37s	remaining: 1m 25s
647:	learn: 0.1630946	total: 2m 37s	remaining: 1m 25s
648:	learn: 0.1628982	total: 2m 37s	remaining: 1m 25s
649:	learn: 0.1626894	total: 2m 37s	remaining: 1m 24s
650:	learn: 0.1624034	total:

788:	learn: 0.1286490	total: 3m 11s	remaining: 51.1s
789:	learn: 0.1286277	total: 3m 11s	remaining: 50.9s
790:	learn: 0.1283504	total: 3m 11s	remaining: 50.6s
791:	learn: 0.1281853	total: 3m 11s	remaining: 50.4s
792:	learn: 0.1281359	total: 3m 12s	remaining: 50.2s
793:	learn: 0.1279312	total: 3m 12s	remaining: 49.9s
794:	learn: 0.1276641	total: 3m 12s	remaining: 49.7s
795:	learn: 0.1275172	total: 3m 12s	remaining: 49.4s
796:	learn: 0.1273312	total: 3m 13s	remaining: 49.2s
797:	learn: 0.1270831	total: 3m 13s	remaining: 48.9s
798:	learn: 0.1270354	total: 3m 13s	remaining: 48.7s
799:	learn: 0.1268022	total: 3m 13s	remaining: 48.5s
800:	learn: 0.1265232	total: 3m 14s	remaining: 48.2s
801:	learn: 0.1261794	total: 3m 14s	remaining: 48s
802:	learn: 0.1259092	total: 3m 14s	remaining: 47.7s
803:	learn: 0.1257415	total: 3m 14s	remaining: 47.5s
804:	learn: 0.1255847	total: 3m 15s	remaining: 47.2s
805:	learn: 0.1254814	total: 3m 15s	remaining: 47s
806:	learn: 0.1252354	total: 3m 15s	remaining: 46.

944:	learn: 0.1000340	total: 3m 48s	remaining: 13.3s
945:	learn: 0.0998816	total: 3m 48s	remaining: 13.1s
946:	learn: 0.0996561	total: 3m 49s	remaining: 12.8s
947:	learn: 0.0994517	total: 3m 49s	remaining: 12.6s
948:	learn: 0.0993837	total: 3m 49s	remaining: 12.3s
949:	learn: 0.0992661	total: 3m 50s	remaining: 12.1s
950:	learn: 0.0990485	total: 3m 50s	remaining: 11.9s
951:	learn: 0.0988431	total: 3m 50s	remaining: 11.6s
952:	learn: 0.0987262	total: 3m 50s	remaining: 11.4s
953:	learn: 0.0984939	total: 3m 50s	remaining: 11.1s
954:	learn: 0.0982517	total: 3m 51s	remaining: 10.9s
955:	learn: 0.0980701	total: 3m 51s	remaining: 10.6s
956:	learn: 0.0979615	total: 3m 51s	remaining: 10.4s
957:	learn: 0.0977665	total: 3m 51s	remaining: 10.2s
958:	learn: 0.0975709	total: 3m 52s	remaining: 9.92s
959:	learn: 0.0973345	total: 3m 52s	remaining: 9.68s
960:	learn: 0.0972040	total: 3m 52s	remaining: 9.44s
961:	learn: 0.0970403	total: 3m 52s	remaining: 9.19s
962:	learn: 0.0967640	total: 3m 53s	remaining:

# Финал

In [31]:
if bm == True and bcm == False:
    fx = X1_fut
    #fx.to_csv("fx.csv", index = False)
    best_model.fit(X1, y1)
    choice_predicted = best_model.predict(fx)
    choosed_model = best_model
elif bm == False and bcm == True:
    fx = X_full_cat
    best_cat_model.fit(X_train_cat, y_train_cat)
    choice_predicted = best_cat_model.predict(fx)
    choosed_model = best_cat_model
elif bm == False and bcm == False:
    fx = X1_fut
    choice_predicted = stacking(X1, fx, y1)
    choosed_model = 'Stacking'

In [32]:
print(choosed_model)

KNeighborsClassifier(metric='manhattan', n_neighbors=2)


In [33]:
prediction = pd.DataFrame(columns = ['Company', 'Ticker', 'Prediction'])
prediction['Company'] = data1[len(full_data1.index)-num_fut1:]['Company']
prediction['Ticker'] = data1[len(full_data1.index)-num_fut1:]['Ticker']
prediction['Prediction'] = choice_predicted
prediction = prediction.reset_index(drop = True)
prediction

,Company,Ticker,Prediction
0,Apple Inc.,AAPL,1.0
1,Microsoft Corporation,MSFT,1.0
2,Amazon.com Inc.,AMZN,1.0
3,Tesla Inc,TSLA,1.0
4,Facebook Inc. Class A,FB,1.0
...,...,...,...
447,TechnipFMC Plc,FTI,0.0
448,HollyFrontier Corporation,HFC,0.0
449,Gap Inc.,GPS,1.0
450,Xerox Holdings Corporation,XRX,0.0


In [34]:
prediction['Prediction'].value_counts()

0.0    306
1.0    146
Name: Prediction, dtype: int64

In [35]:
prediction.to_csv(r'Result/prediction.csv', index = False)

In [36]:
end = time.time()
print(end - start)

7832.223977327347


In [40]:
perpective = prediction[prediction['Prediction'] == 1].reset_index(drop = True)
perpective.to_csv(r'Result/perpective.csv', index = False)